In [3]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [2]:
#install CATBOOST
pip install catboost

     |████████████████████████████████| 62.9MB 38kB/s 


In [111]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
import seaborn as seabornInstance 
from catboost import CatBoostRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error 
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score
import lightgbm as lgb
import xgboost as xgb
# dataset = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ML/Train_Data.csv')
# dataWithoutLabels = pd.read_csv(('/content/drive/My Drive/Colab Notebooks/ML/Test_Data.csv'))
# temp=pd.concat([dataset,dataWithoutLabels], ignore_index=True)
# print(dataset.tail)
# print(temp.columns.tolist())
#temp['Yearly Income in addition to Salary (e.g. Rental Income)']=pd.to_numeric(temp['Yearly Income in addition to Salary (e.g. Rental Income)'].map(lambda x: x.rstrip(' EUR')))
#print(temp['Yearly Income in addition to Salary (e.g. Rental Income)'].unique())

In [112]:
# col='Yearly Income in addition to Salary (e.g. Rental Income)'
# temp[col]=temp[col].map(lambda x: x.rstrip(' EUR'))
# print(temp[col].unique())
# a=temp[col].value_counts(dropna=False)
# print(a)
# X_train,X_test = temp.iloc[:1048574],temp.iloc[1048575:]
# print(X_train.head)
# rename_cols = {"Income in EUR":'Income'}
# train = train.rename(columns=rename_cols)
# for col in fill_col_dict.keys():
    # data[col] = data[col].fillna(fill_col_dict[col])
#X_train,X_test = data[features_col].iloc[:1048574],data[features_col].iloc[1048575:]

In [119]:

# to add noise on the data if dataframe has any NAN values
def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))


#Function to apply Target encoding on categorial data it takes train and test column along with target column to give logical mean value to strings.

def target_encode(trn_series, tst_series,target):
    min_samples_leaf=1
    smoothing=1
    noise_level=0
    temp = pd.concat([trn_series, target], axis=1)
    
    # Compute target mean 
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    
    # Apply average function to all target data
    prior = target.mean()
    
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    
    # Apply averages to train and test series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index 
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    
    return add_noise(ft_trn_series, noise_level), add_noise(ft_tst_series, noise_level)


In [120]:

def prepareData(dataset,dataWithoutLabels):
  #training data+test data
  
  dataset['Gender']=dataset['Gender'].replace(['unknown','0'], np.nan)  
  dataset['Gender']=dataset['Gender'].replace(['f'],'female')  
  dataset['Gender'] = dataset['Gender'].fillna('male')
  dataset['Housing Situation']=dataset['Housing Situation'].replace(['nA',0,'0'], np.nan)
  dataset['Housing Situation'] = dataset['Housing Situation'].fillna(dataset['Housing Situation'].mode()[0]) 
    # dataset['Housing Situation']=dataset['Housing Situation'].replace(['Small House','Small Apartment'],'Small')
  # dataset['Housing Situation']=dataset['Housing Situation'].replace(['Large House','Large Apartment', 'Castle'],'Large')
  # dataset['Housing Situation']=dataset['Housing Situation'].replace(['Medium House','Medium Apartment'],'Medium')
  
  dataset['Work Experience in Current Job [years]']=pd.to_numeric(dataset['Work Experience in Current Job [years]'].replace(['#NUM!'],0))
  dataset['Work Experience in Current Job [years]'] = dataset['Work Experience in Current Job [years]'].fillna(dataset['Work Experience in Current Job [years]'].median())
  #dataset['Satisfation with employer']=dataset['Satisfation with employer'].replace([np.nan],'Average')
  dataset['Satisfation with employer'] = dataset['Satisfation with employer'].fillna(dataset['Satisfation with employer'].mode()[0])
  dataset['Yearly Income in addition to Salary (e.g. Rental Income)']=pd.to_numeric(dataset['Yearly Income in addition to Salary (e.g. Rental Income)'].map(lambda x: x.rstrip(' EUR')))
  dataset['Hair Color']=dataset['Hair Color'].replace(['unknown','0'], np.nan)
  dataset['Year of Record'] = dataset['Year of Record'].fillna(dataset['Year of Record'].median())  
  #dataset['Hair Color']=dataset['Hair Color'].replace([np.nan],'Brown')
  dataset['Hair Color'] = dataset['Hair Color'].fillna(dataset['Hair Color'].mode()[0])
  #dataset['Profession'] = dataset['Profession'].fillna(dataset['Profession'].mode()[0])
  #print('fillna mean')
  dataWithoutLabels['Gender']=dataWithoutLabels['Gender'].replace(['unknown','0'],np.nan)  
  dataWithoutLabels['Gender']=dataWithoutLabels['Gender'].replace(['f'],'female')  
  #dataWithoutLabels['Gender']=dataWithoutLabels['Gender'].replace([np.nan],')
  dataWithoutLabels['Gender'] = dataWithoutLabels['Gender'].fillna('male')                                                               
  
  dataWithoutLabels['Housing Situation']=dataWithoutLabels['Housing Situation'].replace(['nA',0,'0'], np.nan)
  dataWithoutLabels['Housing Situation'] = dataWithoutLabels['Housing Situation'].fillna(dataWithoutLabels['Housing Situation'].mode()[0])
  # dataWithoutLabels['Housing Situation']=dataWithoutLabels['Housing Situation'].replace(['Small House','Small Apartment'],'Small')
  # dataWithoutLabels['Housing Situation']=dataWithoutLabels['Housing Situation'].replace(['Large House','Large Apartment', 'Castle'],'Large')
  # dataWithoutLabels['Housing Situation']=dataWithoutLabels['Housing Situation'].replace(['Medium House','Medium Apartment'],'Medium')

  dataWithoutLabels['Work Experience in Current Job [years]']=pd.to_numeric(dataWithoutLabels['Work Experience in Current Job [years]'].replace(['#NUM!'],0))
  dataWithoutLabels['Work Experience in Current Job [years]'] = dataWithoutLabels['Work Experience in Current Job [years]'].fillna(dataWithoutLabels['Work Experience in Current Job [years]'].median())
  dataWithoutLabels['Satisfation with employer'] = dataWithoutLabels['Satisfation with employer'].fillna(dataWithoutLabels['Satisfation with employer'].mode()[0])  
  dataWithoutLabels['Yearly Income in addition to Salary (e.g. Rental Income)']=pd.to_numeric(dataWithoutLabels['Yearly Income in addition to Salary (e.g. Rental Income)'].map(lambda x: x.rstrip(' EUR')))
  dataWithoutLabels['Hair Color']=dataWithoutLabels['Hair Color'].replace(['unknown','0'],np.nan)
  #dataset['Year of Record']=dataset['Year of Record'].replace([np.nan],'1942')
  dataWithoutLabels['Year of Record'] = dataWithoutLabels['Year of Record'].fillna(dataWithoutLabels['Year of Record'].median())
  dataWithoutLabels['Hair Color'] = dataWithoutLabels['Hair Color'].fillna(dataWithoutLabels['Hair Color'].mode()[0])

  #dataWithoutLabels=dataWithoutLabels.fillna(method='ffill')
  #dataset=dataset.fillna(method='ffill')
    #encoding of categorical data

  #dataset['Housing Situation'],dataWithoutLabels['Housing Situation']=target_encode(dataset['Housing Situation'], dataWithoutLabels['Housing Situation'],dataset['Total Yearly Income [EUR]'])
  #dataset['Satisfation with employer'],dataWithoutLabels['Satisfation with employer']=target_encode(dataset['Satisfation with employer'], dataWithoutLabels['Satisfation with employer'],dataset['Total Yearly Income [EUR]'])
  #dataset['Gender'],dataWithoutLabels['Gender']=target_encode(dataset['Gender'], dataWithoutLabels['Gender'],dataset['Total Yearly Income [EUR]'])
  #dataset['Country'],dataWithoutLabels['Country']=target_encode(dataset['Country'], dataWithoutLabels['Country'],dataset['Total Yearly Income [EUR]'])
  #dataset['Profession'],dataWithoutLabels['Profession']=target_encode(dataset['Profession'], dataWithoutLabels['Profession'],dataset['Total Yearly Income [EUR]'])
  #dataset['University Degree'],dataWithoutLabels['University Degree']=target_encode(dataset['University Degree'], dataWithoutLabels['University Degree'],dataset['Total Yearly Income [EUR]'])
  #dataset['Hair Color'],dataWithoutLabels['Hair Color']=target_encode(dataset['Hair Color'], dataWithoutLabels['Hair Color'],dataset['Total Yearly Income [EUR]'])
  print(dataset.isna().sum())
  print(dataWithoutLabels.isna().sum())  
  return dataset,dataWithoutLabels


In [121]:

  # plot the Values on a graph
def plotActualVsPredictedIncome(y_test, y_pred):
    df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
    df1 = df.head(25)
    df1.plot(kind='bar',figsize=(10,8))
    plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
    plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
    plt.show()    

def trainModel(X_train, y_train, X_test, y_test):
   
    
    #Implementing linear regression (error rate 110000+)
    # trainingRegressor = LinearRegression()  
    # trainingRegressor.fit(X_train, y_train)
    # y_pred = trainingRegressor.predict(X_test)
    
    
    #Implementing RandomForest Regressor (error rate 80000+)
    #trainingRegressor = RandomForestRegressor(n_estimators=100)
    #trainingRegressor.fit(X_train,y_train)
    #y_pred = trainingRegressor.predict(X_test)
    
    #Implementing lightgbm    
    trainingRegressor = lgb.LGBMRegressor(num_leaves=400,
                                          learning_rate=0.05,
                                          n_estimators=10000)
    trainingRegressor.fit(X_train, y_train,
                          eval_set=[(X_test, y_test)],
                          eval_metric='l1',
                          early_stopping_rounds=5)


    
    
    print('Starting predicting...')
    # predict
    y_pred = trainingRegressor.predict(X_test, num_iteration=trainingRegressor.best_iteration_)
    
    
    #Implementing caatboost (error rate 60000+)
    #learning rate to get an optimal solution 
    
    # trainingRegressor = CatBoostRegressor(iterations=7000,learning_rate=0.02)
    # trainingRegressor.fit(X_train, y_train,use_best_model=True,verbose=True)
    # y_pred=trainingRegressor.predict(X_test)
    
    
    #evaluating the error
    df = pd.DataFrame({'Test': y_test, 'Prediction': y_pred})
    print("mean_absolute_error: %.2f"
      % mean_absolute_error(y_test, y_pred))
    
    #ploting the values on the graph
    #plotActualVsPredictedIncome(y_test, y_pred)

    return trainingRegressor
 

def runTrainedModelOnActualData(regressor, featuresToConsider,dataWithoutLabels, encoder):
    print(featuresToConsider)
    print('dataset',dataWithoutLabels[featuresToConsider].dtypes)
    #get the values for X_test    
    
    X = dataWithoutLabels[featuresToConsider]
    X = encoder.transform(X)
    print('head',X)
    #df = pd.DataFrame(X)
    # scaler = StandardScaler()
    
    
    
    #scaler = sc.transform(df[['Year of Record', 'Crime Level in the City of Employement', 'Age', 'Size of City','Profession','Body Height [cm]', 'Yearly Income in addition to Salary (e.g. Rental Income)']])
    X = sc.transform(X)
    #df['Year of Record'] = scaler[0]
    #df['Crime Level in the City of Employement'] = scaler[1]
    #df['Age'] = scaler[2]
    #df['Size of City'] = scaler[3]
    #df['Profession'] = scaler[4]
    #df['Body Height [cm]'] = scaler[5]
    #df['Yearly Income in addition to Salary (e.g. Rental Income)'] = scaler[6]
    #X = df
    #prediciting the values on the real test data
    predictedData = regressor.predict(X, num_iteration=regressor.best_iteration_)
    # predictedData_antilog = np.exp(predictedData)
    predictedData = np.exp(predictedData)
    df = pd.DataFrame({'Predicted': predictedData})

    #write the data to csv
    # fileName = '/content/drive/My Drive/Colab Notebooks/ML/Sumission_file.csv'
    # df.to_csv(fileName, sep=',',index=False)
    
    dataset1 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ML/tcd-ml-1920-group-income-submission.csv')
    print("predicted",predictedData.shape)
    print("dataWithoutLabels",dataWithoutLabels.shape)
    print("dataset",dataset1.shape)
    dataset1['Instance'] = dataWithoutLabels['Instance']
    dataset1['Total Yearly Income [EUR]'] = predictedData
    dataset1.to_csv('/content/drive/My Drive/Colab Notebooks/ML/Sumission_file_new.csv', index=False)
    

    

#Reading data from CSV
dataset = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ML/Train_Data.csv')
dataWithoutLabels = pd.read_csv(('/content/drive/My Drive/Colab Notebooks/ML/Test_Data.csv'))
#combined_data=pd.concat([dataset,dataWithoutLabels], ignore_index=True)

#preprocessing data on both the datasets



C:\Users\Spy\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Spy\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [122]:

dataset,dataWithoutLabels = prepareData(dataset,dataWithoutLabels)


Instance                                                        0
Year of Record                                                  0
Housing Situation                                               0
Crime Level in the City of Employement                          0
Work Experience in Current Job [years]                          0
Satisfation with employer                                       0
Gender                                                          0
Age                                                             0
Country                                                         0
Size of City                                                    0
Profession                                                   2853
University Degree                                           80600
Wears Glasses                                                   0
Hair Color                                                      0
Body Height [cm]                                                0
Yearly Inc

In [123]:
featuresToConsider = ['Year of Record', 'Housing Situation', 'Crime Level in the City of Employement', 'Work Experience in Current Job [years]', 'Satisfation with employer', 'Gender', 'Age', 'Country', 'Size of City', 'Profession', 'University Degree', 'Body Height [cm]', 'Yearly Income in addition to Salary (e.g. Rental Income)']

#getting thte target data
y = dataset['Total Yearly Income [EUR]'].values
#Normalization
#dataset[featuresToConsider] = pd.DataFrame(scaler.fit_transform(dataset[featuresToConsider]), columns=featuresToConsider)
#dataWithoutLabels[featuresToConsider] = pd.DataFrame(scaler.fit_transform(dataWithoutLabels[featuresToConsider]), columns=featuresToConsider)
# print(dataset[featuresToConsider].dtypes)
# print(dataset[featuresToConsider].dtypes)
# print(dataset['Year of Record'].unique())

In [124]:

#Features Dataset
X = dataset[featuresToConsider]
import category_encoders as ce
target_encoder = ce.TargetEncoder(cols=['Housing Situation', 'Satisfation with employer', 'Gender', 'Country', 'Profession', 'University Degree'])
target_encoder.fit(X, y)
X = target_encoder.transform(X)


df = pd.DataFrame(X)
sc = preprocessing.MinMaxScaler()
X = sc.fit_transform(X)
#scaler = np.transpose(scaler)
#df['Year of Record'] = scaler[0]
#df['Crime Level in the City of Employement'] = scaler[1]
#df['Age'] = scaler[2]
#df['Size of City'] = scaler[3]
#df['Profession'] = scaler[4]
#df['Body Height [cm]'] = scaler[5]
#df['Yearly Income in addition to Salary (e.g. Rental Income)'] = scaler[6]
#X = df
y = np.log(y)

#divide the training data into test and train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# y_train_log = np.log(y_train)

#get the trained model
regressor = trainModel(X_train, y_train, X_test, y_test)
# print(dataWithoutLabels.head)
# print(dataWithoutLabels.isnull().sum())
#run on the real test data using the. trained model
runTrainedModelOnActualData(regressor, featuresToConsider,dataWithoutLabels, target_encoder)

[1]	valid_0's l1: 1.40296	valid_0's l2: 2.81074
Training until validation scores don't improve for 5 rounds
[2]	valid_0's l1: 1.33489	valid_0's l2: 2.54774
[3]	valid_0's l1: 1.27022	valid_0's l2: 2.30993
[4]	valid_0's l1: 1.20888	valid_0's l2: 2.09522
[5]	valid_0's l1: 1.15062	valid_0's l2: 1.9012
[6]	valid_0's l1: 1.09525	valid_0's l2: 1.72581
[7]	valid_0's l1: 1.04283	valid_0's l2: 1.56731
[8]	valid_0's l1: 0.993019	valid_0's l2: 1.42401
[9]	valid_0's l1: 0.945726	valid_0's l2: 1.29451
[10]	valid_0's l1: 0.900909	valid_0's l2: 1.17743
[11]	valid_0's l1: 0.858378	valid_0's l2: 1.07159
[12]	valid_0's l1: 0.818068	valid_0's l2: 0.975869
[13]	valid_0's l1: 0.779813	valid_0's l2: 0.889187
[14]	valid_0's l1: 0.743599	valid_0's l2: 0.810908
[15]	valid_0's l1: 0.709217	valid_0's l2: 0.739948
[16]	valid_0's l1: 0.676657	valid_0's l2: 0.675838
[17]	valid_0's l1: 0.645756	valid_0's l2: 0.617774
[18]	valid_0's l1: 0.616434	valid_0's l2: 0.565191
[19]	valid_0's l1: 0.588706	valid_0's l2: 0.517611

[161]	valid_0's l1: 0.124723	valid_0's l2: 0.0361874
[162]	valid_0's l1: 0.124682	valid_0's l2: 0.0361556
[163]	valid_0's l1: 0.124593	valid_0's l2: 0.036111
[164]	valid_0's l1: 0.124502	valid_0's l2: 0.0360636
[165]	valid_0's l1: 0.124418	valid_0's l2: 0.0360288
[166]	valid_0's l1: 0.124334	valid_0's l2: 0.035994
[167]	valid_0's l1: 0.124269	valid_0's l2: 0.0359654
[168]	valid_0's l1: 0.124192	valid_0's l2: 0.0359322
[169]	valid_0's l1: 0.124106	valid_0's l2: 0.0358921
[170]	valid_0's l1: 0.124037	valid_0's l2: 0.0358593
[171]	valid_0's l1: 0.123965	valid_0's l2: 0.0358268
[172]	valid_0's l1: 0.123898	valid_0's l2: 0.0357984
[173]	valid_0's l1: 0.123802	valid_0's l2: 0.035743
[174]	valid_0's l1: 0.123728	valid_0's l2: 0.0357121
[175]	valid_0's l1: 0.123669	valid_0's l2: 0.0356845
[176]	valid_0's l1: 0.12355	valid_0's l2: 0.035622
[177]	valid_0's l1: 0.123509	valid_0's l2: 0.0356039
[178]	valid_0's l1: 0.123447	valid_0's l2: 0.0355771
[179]	valid_0's l1: 0.123387	valid_0's l2: 0.035549

[319]	valid_0's l1: 0.119262	valid_0's l2: 0.0338842
[320]	valid_0's l1: 0.119237	valid_0's l2: 0.0338712
[321]	valid_0's l1: 0.119227	valid_0's l2: 0.033869
[322]	valid_0's l1: 0.119201	valid_0's l2: 0.0338587
[323]	valid_0's l1: 0.119181	valid_0's l2: 0.033854
[324]	valid_0's l1: 0.11917	valid_0's l2: 0.0338495
[325]	valid_0's l1: 0.119156	valid_0's l2: 0.0338451
[326]	valid_0's l1: 0.119147	valid_0's l2: 0.0338422
[327]	valid_0's l1: 0.119136	valid_0's l2: 0.0338392
[328]	valid_0's l1: 0.119123	valid_0's l2: 0.0338364
[329]	valid_0's l1: 0.119108	valid_0's l2: 0.0338313
[330]	valid_0's l1: 0.1191	valid_0's l2: 0.0338278
[331]	valid_0's l1: 0.119087	valid_0's l2: 0.033824
[332]	valid_0's l1: 0.11908	valid_0's l2: 0.0338202
[333]	valid_0's l1: 0.119062	valid_0's l2: 0.0338152
[334]	valid_0's l1: 0.11905	valid_0's l2: 0.0338112
[335]	valid_0's l1: 0.119021	valid_0's l2: 0.0338038
[336]	valid_0's l1: 0.11901	valid_0's l2: 0.0338011
[337]	valid_0's l1: 0.118986	valid_0's l2: 0.0337936
[3

[476]	valid_0's l1: 0.117455	valid_0's l2: 0.0332989
[477]	valid_0's l1: 0.117446	valid_0's l2: 0.033296
[478]	valid_0's l1: 0.117436	valid_0's l2: 0.0332937
[479]	valid_0's l1: 0.117425	valid_0's l2: 0.0332917
[480]	valid_0's l1: 0.117421	valid_0's l2: 0.033291
[481]	valid_0's l1: 0.117415	valid_0's l2: 0.0332888
[482]	valid_0's l1: 0.11741	valid_0's l2: 0.0332871
[483]	valid_0's l1: 0.1174	valid_0's l2: 0.0332845
[484]	valid_0's l1: 0.11739	valid_0's l2: 0.0332832
[485]	valid_0's l1: 0.117388	valid_0's l2: 0.0332823
[486]	valid_0's l1: 0.117382	valid_0's l2: 0.0332807
[487]	valid_0's l1: 0.117374	valid_0's l2: 0.0332778
[488]	valid_0's l1: 0.117356	valid_0's l2: 0.0332734
[489]	valid_0's l1: 0.117345	valid_0's l2: 0.0332707
[490]	valid_0's l1: 0.117337	valid_0's l2: 0.0332682
[491]	valid_0's l1: 0.117314	valid_0's l2: 0.033255
[492]	valid_0's l1: 0.117303	valid_0's l2: 0.0332509
[493]	valid_0's l1: 0.117294	valid_0's l2: 0.0332473
[494]	valid_0's l1: 0.117287	valid_0's l2: 0.033246
[

[633]	valid_0's l1: 0.116299	valid_0's l2: 0.0329391
[634]	valid_0's l1: 0.116292	valid_0's l2: 0.0329372
[635]	valid_0's l1: 0.116286	valid_0's l2: 0.0329357
[636]	valid_0's l1: 0.116283	valid_0's l2: 0.0329349
[637]	valid_0's l1: 0.116279	valid_0's l2: 0.0329337
[638]	valid_0's l1: 0.116277	valid_0's l2: 0.0329333
[639]	valid_0's l1: 0.116268	valid_0's l2: 0.0329315
[640]	valid_0's l1: 0.116261	valid_0's l2: 0.032929
[641]	valid_0's l1: 0.116254	valid_0's l2: 0.0329272
[642]	valid_0's l1: 0.116249	valid_0's l2: 0.032926
[643]	valid_0's l1: 0.116245	valid_0's l2: 0.0329247
[644]	valid_0's l1: 0.116243	valid_0's l2: 0.0329232
[645]	valid_0's l1: 0.116234	valid_0's l2: 0.0329181
[646]	valid_0's l1: 0.116228	valid_0's l2: 0.0329166
[647]	valid_0's l1: 0.116223	valid_0's l2: 0.0329157
[648]	valid_0's l1: 0.116208	valid_0's l2: 0.0329088
[649]	valid_0's l1: 0.1162	valid_0's l2: 0.0329065
[650]	valid_0's l1: 0.116195	valid_0's l2: 0.032905
[651]	valid_0's l1: 0.116192	valid_0's l2: 0.032904

[793]	valid_0's l1: 0.115487	valid_0's l2: 0.0327062
[794]	valid_0's l1: 0.115485	valid_0's l2: 0.0327063
[795]	valid_0's l1: 0.11548	valid_0's l2: 0.0327047
[796]	valid_0's l1: 0.115472	valid_0's l2: 0.0327021
[797]	valid_0's l1: 0.115468	valid_0's l2: 0.0327018
[798]	valid_0's l1: 0.115467	valid_0's l2: 0.0327002
[799]	valid_0's l1: 0.115468	valid_0's l2: 0.0327004
[800]	valid_0's l1: 0.115463	valid_0's l2: 0.0326983
[801]	valid_0's l1: 0.115461	valid_0's l2: 0.0326977
[802]	valid_0's l1: 0.115453	valid_0's l2: 0.0326961
[803]	valid_0's l1: 0.115448	valid_0's l2: 0.0326945
[804]	valid_0's l1: 0.115444	valid_0's l2: 0.0326931
[805]	valid_0's l1: 0.115438	valid_0's l2: 0.0326911
[806]	valid_0's l1: 0.115431	valid_0's l2: 0.0326891
[807]	valid_0's l1: 0.115426	valid_0's l2: 0.0326852
[808]	valid_0's l1: 0.115422	valid_0's l2: 0.0326842
[809]	valid_0's l1: 0.115404	valid_0's l2: 0.0326765
[810]	valid_0's l1: 0.115399	valid_0's l2: 0.0326753
[811]	valid_0's l1: 0.115394	valid_0's l2: 0.03

[952]	valid_0's l1: 0.114904	valid_0's l2: 0.0325209
[953]	valid_0's l1: 0.114902	valid_0's l2: 0.032521
[954]	valid_0's l1: 0.114902	valid_0's l2: 0.0325225
[955]	valid_0's l1: 0.1149	valid_0's l2: 0.0325224
[956]	valid_0's l1: 0.114898	valid_0's l2: 0.0325223
[957]	valid_0's l1: 0.114895	valid_0's l2: 0.0325218
Early stopping, best iteration is:
[952]	valid_0's l1: 0.114904	valid_0's l2: 0.0325209
Starting predicting...
mean_absolute_error: 0.11
['Year of Record', 'Housing Situation', 'Crime Level in the City of Employement', 'Work Experience in Current Job [years]', 'Satisfation with employer', 'Gender', 'Age', 'Country', 'Size of City', 'Profession', 'University Degree', 'Body Height [cm]', 'Yearly Income in addition to Salary (e.g. Rental Income)']
dataset Year of Record                                              float64
Housing Situation                                            object
Crime Level in the City of Employement                        int64
Work Experience in Curre

predicted (369438,)
dataWithoutLabels (369438, 17)
dataset (369438, 2)
